# 🚀 Messung der OCR-Qualität in Python

In diesem Notebook bewerten wir die OCR-Qualität. 

## Installationen und Importe

In [1]:
# 🚀 Install libraries  
!sudo apt install tesseract-ocr
!sudo apt install tesseract-ocr-frk
!sudo apt install poppler-utils
!pip install pytesseract pillow Levenshtein

'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pytesseract
from PIL import Image
from pathlib import Path

## for interactivity in jupyter books
from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import CustomJS, TextInput

## Durchführen von OCR

Wir werden denselben Beispielartikel zur Bewertung verwenden:
<!-- In this notebook we evaluate OCR quality. We'll use the same sample article for evaluation: -->

<img src='grippe.jpeg' width="600">

In [3]:
# 🚀 get the image to process in Colab
!wget https://raw.githubusercontent.com/quadriga-dk/Text-Fallstudie-1/refs/heads/main/ocr/grippe.jpeg

'wget' is not recognized as an internal or external command,
operable program or batch file.


Zuerst führen wir die OCR durch: <!-- Let us OCR it first: -->

In [11]:
#ocr_output = pytesseract.image_to_string(Image.open('grippe.jpeg'), lang='frk') # Wir verwenden das Fraktur-OCR-Modell
# 🚀 get the image to process in Colab
image_path = r"..\assets\images\grippe.jpeg"
img = Image.open(image_path)
ocr_output = pytesseract.image_to_string(img, lang='deu_latf') # Wir verwenden das Fraktur-OCR-Modell

In [12]:
print(ocr_output)

Zie Grippe wüfel weiter

Zunahme der ſchweren Fälle in Berlin.

Die Zahl der Grippefälle iſt in den letzten
beider Tagen auch in Groß-Berlin noH
erfblih zefitiegen. Die Worenhäuſer und ſon-
Haen aroßen GerſHäfte, die Krirgs- und die pri«
n Betriebe lagen, daß übermäig viele An«

: fich 5cben kren? melden miüen,-und an:
; ew Loſt und 5ei der Straßenbahn iſt der
ſoz der Grippekranken bedeuten) g&



### Manuell die 'Ground Truth' zur Bewertung erstellen <!-- ### Manually create  the 'ground truth' to evaluate against -->

In [13]:
output_notebook()

capture_ground_truth = TextInput(value="""Die Grippe wütet weiter. Zunahme der schweren Fälle in Berlin. Die Zahl der Grippefälle ist in den letzten Tagen auch in Groß-Berlin noch erheblich gestiegen. Die Warenhäuser und sonstigen großen Geschäfte, die Kriegs- und die privaten Betriebe klagen, daß übermäßig viele Angestellte sich haben krank melden müssen und auch bei der Post und bei der Straßenbahn ist der Prozentsatz der Grippekranken deutlich gestiegen.""", 
                                 title="Please insert corrected string: ") 

ground_truth = capture_ground_truth.value.strip()

rewrite_var_after_input = CustomJS(args=dict(text_input=capture_ground_truth), code="""
    var word = text_input.value.trim();
    console.log('Input value:', word);
    function sendToPython(){
    var kernel = IPython.notebook.kernel;
    kernel.execute("ground_truth = '" + word + "'");
    }
    sendToPython();
""")



capture_ground_truth.js_on_change('value', rewrite_var_after_input)

Loading BokehJS ...

In [14]:
# Layout and display
layout = column(capture_ground_truth)

show(layout)

In [15]:
print(ground_truth)

Die Grippe wütet weiter. Zunahme der schweren Fälle in Berlin. Die Zahl der Grippefälle ist in den letzten Tagen auch in Groß-Berlin noch erheblich gestiegen. Die Warenhäuser und sonstigen großen Geschäfte, die Kriegs- und die privaten Betriebe klagen, daß übermäßig viele Angestellte sich haben krank melden müssen und auch bei der Post und bei der Straßenbahn ist der Prozentsatz der Grippekranken deutlich gestiegen.


## Berechnung von Precision, Recall und F1-Measure <!-- ### Calculate precision, recall, and F1-measure -->

In [16]:
# 🚀 create folder and get the auxiliary python script to run in Colab
aux_dir = Path("auxiliary")
if not aux_dir.exists():
    aux_dir.mkdir(parents=True)

In [17]:
# 🚀 create folder and get the auxiliary python script to run in Colab
!wget https://raw.githubusercontent.com/quadriga-dk/Text-Fallstudie-1/refs/heads/main/data-input/auxiliary/measure_ocr_quality.py
!mv measure_ocr_quality.py auxiliary/

'wget' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
from auxiliary.measure_ocr_quality import measure_ocr_quality

In [19]:
precision, recall, f_score = measure_ocr_quality(ocr_output, ground_truth)

In [20]:
print(f'Precision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1-score: {round(f_score, 4)}')

Precision: 0.79
Recall: 0.8317
F1-score: 0.8103


### 🚀 Compare Ground Truth with OCR Output

The block below lets you explore how **character-level OCR mistakes** affect **Precision**, **Recall**, and the **F1-score**.

**How to use it:**

1. The *OCR Output* box is already filled with the text produced by the OCR system.  
2. Copy the **ground truth** from above into the *Ground Truth* box.
3. Click **Compare**.

The visualization highlights differences character by character:
- 🟩 **green** = correctly recognized (TP)  
- 🟥 **red** = extra or incorrect characters (FP)  
- 🟦 **blue** = missing characters (FN)

Below the visualization, the tool computes **Precision**, **Recall**, and **F1**  directly from these TP/FP/FN values.

This helps illustrate how different types of OCR errors influence each metric.

Changing the Ground Truth will also change the metrics. Go ahead and play with the GT and see how the metrics change

```{raw} html
<div id="diffTool" style="font-family:system-ui; max-width:800px;">
  <label><strong>Ground Truth:</strong></label>
  <textarea id="gtText" rows="3" style="width:100%;padding:6px;font-family:monospace;"></textarea>

  <br><br>

  <label><strong>OCR Output:</strong></label>
  <textarea id="ocrText" rows="3" style="width:100%;padding:6px;font-family:monospace;"></textarea>

  <br><br>

  <button id="compareBtn"
          style="padding:6px 14px;background:#333;color:white;border:none;border-radius:4px;">
    Compare
  </button>

  <div id="legend" style="margin-top:12px; font-size:0.9rem;">
      <strong>Legend:</strong><br>
      <span style="background:#2ecc71; color:black; padding:1px 4px;">correct</span> &nbsp;
      <span style="background:#e74c3c; color:white; padding:1px 4px;">FP</span> (wrong/extra) &nbsp;
      <span style="background:#3498db; color:white; padding:1px 4px;">FN</span> (missing)
  </div>

  <div id="diffResult" style="margin-top:20px;font-family:monospace;white-space:pre-wrap;"></div>
  <div id="metricsOut" style="margin-top:15px;line-height:1.5;"></div>
  
</div>

<script>
window.addEventListener("load", function(){

    // -------------------------------
    // AUTO-LOAD OCR OUTPUT (robust version)
    // -------------------------------
    (function autoFillOCR(){
        const ocrBox = document.getElementById("ocrText");
        if (!ocrBox) return;
    
        const pres = document.querySelectorAll("pre");
    
        let found = "";
        for (let p of pres) {
    
            const txt = p.innerText.trim();
    
            // Skip empty or tiny blocks
            if (txt.length < 30) continue;
    
            // Skip code-like blocks
            if (txt.includes("import") ||
                txt.includes("#") ||
                txt.includes("=") ||
                txt.includes("Image.open") ||
                txt.includes("pytesseract")) {
                continue;
            }
    
            // Prefer multi-line natural OCR text
            if (txt.split("\n").length > 2) {
                found = txt;
                break;
            }
        }
    
        if (found) {
            console.log("Auto-loaded REAL OCR text.");
            ocrBox.value = found;
        } else {
            console.log("No valid OCR text found.");
        }
    })();


    console.log("IN IFRAME: JS loaded.");

    const btn = document.getElementById("compareBtn");
    const gtBox = document.getElementById("gtText");
    const ocrBox = document.getElementById("ocrText");

    if (!btn || !gtBox || !ocrBox) {
        console.log("Elements NOT found inside iframe");
        return;
    }

    function diffChars(oldStr, newStr){
        const oldArr = oldStr.split("");
        const newArr = newStr.split("");
        const dp = Array(oldArr.length+1).fill().map(()=>Array(newArr.length+1).fill(0));

        for(let i=0;i<=oldArr.length;i++) dp[i][0]=i;
        for(let j=0;j<=newArr.length;j++) dp[0][j]=j;

        for(let i=1;i<=oldArr.length;i++){
            for(let j=1;j<=newArr.length;j++){
                if(oldArr[i-1]===newArr[j-1]) dp[i][j]=dp[i-1][j-1];
                else dp[i][j]=Math.min(
                    dp[i-1][j]+1,
                    dp[i][j-1]+1,
                    dp[i-1][j-1]+1
                );
            }
        }

        let i=oldArr.length, j=newArr.length, diffs=[];
        while(i>0 || j>0){
            if(i>0&&j>0&&oldArr[i-1]===newArr[j-1]){
                diffs.push({value:oldArr[i-1]});
                i--; j--;
            } else if(j>0 && dp[i][j]===dp[i][j-1]+1){
                diffs.push({value:newArr[j-1], added:true});
                j--;
            } else if(i>0 && dp[i][j]===dp[i-1][j]+1){
                diffs.push({value:oldArr[i-1], removed:true});
                i--;
            } else {
                diffs.push({value:newArr[j-1], added:true});
                diffs.push({value:oldArr[i-1], removed:true});
                i--; j--;
            }
        }

        return diffs.reverse();
    }

    btn.onclick = function(){

        console.log("Button clicked inside iframe");

        const gt = gtBox.value;
        const ocr = ocrBox.value;

        const diff = diffChars(gt, ocr);

        let TP=0, FP=0, FN=0;
        let html="";

        diff.forEach(p=>{
            if(p.added){ FP++; html+=`<span style='background:#e74c3c;color:white;'>${p.value}</span>`; }
            else if(p.removed){ FN++; html+=`<span style='background:#3498db;color:white;'>␣</span>`; }
            else { TP++; html+=`<span style='background:#2ecc71;color:black;'>${p.value}</span>`; }
        });

        document.getElementById("diffResult").innerHTML = html;

        const precision = TP+FP===0?0:TP/(TP+FP);
        const recall = TP+FN===0?0:TP/(TP+FN);
        const f1 = (precision+recall===0)?0:2*(precision*recall)/(precision+recall);

        document.getElementById("metricsOut").innerHTML =
          `<strong>TP:</strong> ${TP} &nbsp;
           <strong>FP:</strong> ${FP} &nbsp;
           <strong>FN:</strong> ${FN}<br>
           <strong>Precision:</strong> ${precision.toFixed(3)}<br>
           <strong>Recall:</strong> ${recall.toFixed(3)}<br>
           <strong>F1:</strong> ${f1.toFixed(3)}`;
    };

});
</script>
```

🟩 TP increases → Precision ↑ and Recall ↑  
🟥 FP increases → Precision ↓  
🟦 FN increases → Recall ↓